# Proyecto SQL

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

# Tabla de contenidos <a id='back'></a>

1. [Conexión a la base de datos y análisis de tablas](#descarga_datos)
2. [Resolución de ejercicios](#eda)
3. [Conclusiones generales y recomendaciones](#conclusiones)

![Diagrama de datos](https://practicum-content.s3.us-west-1.amazonaws.com/resources/moved_Untitled_-_2020-07-02T142019.920_1593688954.png "Diagrama de datos")

## Conexión a la base de datos y análisis de tablas

En este paso, se hara la conexion y primer analisis de las tablas

In [1]:
# Librerias
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [2]:
# Importamos las variables de entorno
load_dotenv()

True

In [3]:
# Conexion a la base de datos
db_config = {'user': os.getenv('tripleten_sql_user'), # nombre de usuario
             'pwd': os.getenv('tripleten_sql_pwd'), # contraseña
             'host': os.getenv('tripleten_sql_host'),
             'port': 6432, # puerto de conexión
             'db': os.getenv('tripleten_sql_db')} # nombre de la base de datos
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# Vemos el contenido de la tabla books
query = '''
SELECT
    *
FROM
    books
LIMIT
    5
'''
data_books = pd.io.sql.read_sql(query, con=engine)
data_books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# Verficamos que no haya datos nulos
query = ''' 
SELECT
    COUNT(*)
FROM
    books
WHERE
    book_id IS NULL
    OR author_id IS NULL
    OR title IS NULL
    OR num_pages IS NULL
    OR publication_date IS NULL
    OR publisher_id IS NULL;
'''
data_books_null_count = pd.io.sql.read_sql(query, con=engine)
data_books_null_count

,count
0,0


In [6]:
# Verificamos que no haya datos duplicados
query = '''
SELECT
    COUNT(*)
FROM
    (
        SELECT
            book_id,
            author_id,
            title,
            num_pages,
            publication_date,
            publisher_id,
            COUNT(*)
        FROM
            books
        GROUP BY
            book_id,
            author_id,
            title,
            num_pages,
            publication_date,
            publisher_id
        HAVING
            COUNT(*) > 1
    ) AS SUB
'''
data_books_duplicate_count = pd.io.sql.read_sql(query, con=engine)
data_books_duplicate_count

,count
0,0


Nuestra tabla `books` que son los datos sobre libros, contiene las siguientes columnas:

- `book_id`: identificación del libro
- `author_id`: identificación del autor o autora
- `title`: título
- `num_pages`: número de páginas
- `publication_date`: fecha de la publicación
- `publisher_id`: identificación de la editorial

Además de ello, la tabla no presenta datos ausentes o duplicados

In [7]:
# Vemos el contenido de la tabla authors
query = '''
SELECT
    *
FROM
    authors
LIMIT
    5
'''
data_authors = pd.io.sql.read_sql(query, con=engine)
data_authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
# Verficamos que no haya datos nulos
query = '''
SELECT
    COUNT(*)
FROM
    authors
WHERE
    author_id IS NULL
    OR author IS NULL;
'''
data_authors_null_count = pd.io.sql.read_sql(query, con=engine)
data_authors_null_count

,count
0,0


In [9]:
# Verificamos que no haya datos duplicados
query = '''
SELECT
    COUNT(*)
FROM
    (
        SELECT
            author_id,
            author,
            COUNT(*)
        FROM
            authors
        GROUP BY
            author_id,
            author
        HAVING
            COUNT(*) > 1
    ) AS SUB
'''
data_authors_duplicate_count = pd.io.sql.read_sql(query, con=engine)
data_authors_duplicate_count

,count
0,0


Nuestra tabla `authors` que son los datos sobre autores, contiene las siguientes columnas:

- `author_id`: identificación del autor o autora
- `author`: el autor o la autora

Además de ello, la tabla no presenta datos ausentes o duplicados

In [10]:
# Vemos el contenido de la tabla ratings
query = '''
SELECT
    *
FROM
    ratings
LIMIT
    5 '''
data_ratings = pd.io.sql.read_sql(query, con=engine)
data_ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
# Verficamos que no haya datos nulos
query = '''
SELECT
    COUNT(*)
FROM
    ratings
WHERE
    rating_id IS NULL
    OR book_id IS NULL
    OR username IS NULL
    OR rating IS NULL;
 '''
data_ratings_null_count = pd.io.sql.read_sql(query, con=engine)
data_ratings_null_count

,count
0,0


In [12]:
# Verificamos que no haya datos duplicados
query = '''
SELECT
    COUNT(*)
FROM
  (SELECT rating_id,
          book_id,
          username,
          rating,
          COUNT(*)
   FROM
    ratings
   GROUP BY rating_id,
            book_id,
            username,
            rating
   HAVING COUNT(*) > 1
  ) AS SUB
'''
data_ratings_duplicate_count = pd.io.sql.read_sql(query, con=engine)
data_ratings_duplicate_count

,count
0,0


Nuestra tabla `ratings` que son los datos sobre las calificaciones de usuarios, contiene las siguientes columnas:

- `rating_id`: identificación de la calificación
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `rating`: calificación

Además de ello, la tabla no presenta datos ausentes o duplicados

In [13]:
# Vemos el contenido de la tabla reviews
query = '''
SELECT
    *
FROM
    reviews
LIMIT
    5
'''
data_reviews = pd.io.sql.read_sql(query, con=engine)
data_reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [14]:
# Verficamos que no haya datos nulos
query = '''
SELECT
    COUNT(*)
FROM
    reviews
WHERE
    review_id IS NULL
    OR book_id IS NULL
    OR username IS NULL
    OR text IS NULL;
'''
data_reviews_null_count = pd.io.sql.read_sql(query, con=engine)
data_reviews_null_count

,count
0,0


In [15]:
# Verificamos que no haya datos duplicados
query = '''
SELECT
    COUNT(*)
FROM
  (SELECT review_id,
          book_id,
          username, text, COUNT(*)
   FROM
    reviews
   GROUP BY review_id,
            book_id,
            username, text
   HAVING
    COUNT(*) > 1
  ) AS SUB
'''
data_reviews_duplicate_count = pd.io.sql.read_sql(query, con=engine)
data_reviews_duplicate_count

,count
0,0


Nuestra tabla `reviews` que contiene datos sobre las reseñas de los y las clientes, presenta las siguientes columnas:

- `review_id`: identificación de la reseña
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `text`: el texto de la reseña

Además de ello, la tabla no presenta datos ausentes o duplicados

In [16]:
# Vemos el contenido de la tabla publishers
query = '''
SELECT
    *
FROM
    publishers
LIMIT
    5
'''
data_publishers = pd.io.sql.read_sql(query, con=engine)
data_publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [17]:
# Verficamos que no haya datos nulos
query = '''
SELECT
    COUNT(*)
FROM
    publishers
WHERE
    publisher_id IS NULL
    OR publisher IS NULL;
'''
data_publishers_null_count = pd.io.sql.read_sql(query, con=engine)
data_publishers_null_count

,count
0,0


In [18]:
# Verificamos que no haya datos duplicados
query = '''
SELECT
    COUNT(*)
FROM
  (SELECT publisher_id,
          publisher,
          COUNT(*)
   FROM
    publishers
   GROUP BY publisher_id,
            publisher
   HAVING
    COUNT(*) > 1) AS SUB
'''
data_publishers_duplicate_count = pd.io.sql.read_sql(query, con=engine)
data_publishers_duplicate_count

,count
0,0


Nuestra tabla `publishers` que contiene datos sobre editoriales, presenta las siguientes columnas:

- `publisher_id`: identificación de la editorial
- `publisher`: la editorial

Además de ello, la tabla no presenta datos ausentes o duplicados

## Resolución de ejercicios

1. Encuentra el número de libros publicados después del 1 de enero de 2000.
2. Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
3. Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
4. Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
5. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

### Encuentra el número de libros publicados después del 1 de enero de 2000.

In [19]:
# Número de libros publicados después del 1 de enero del 2000
query = '''
SELECT
    COUNT(*) AS cnt
FROM
    BOOKS
WHERE
    publication_date > '2000-01-01'
'''
exercise1 = pd.io.sql.read_sql(query, con=engine)
exercise1

,cnt
0,819


Podemos ver que el número de libros publicados despues del 1 de enero del 2000 son 819

### Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [20]:
# Número de reseñas de usuarios y la calificacion promedio de cada libro
query = '''
SELECT id,
       book_name,
       mean_rating,
       count_reviews
FROM
  (SELECT ratings.book_id AS id,
          books.title AS book_name,
          AVG(ratings.rating) AS mean_rating
   FROM ratings
   INNER JOIN books ON books.book_id = ratings.book_id
   GROUP BY id,
            book_name) AS SUB1
LEFT JOIN
  (SELECT book_id,
          COUNT(username) AS count_reviews
   FROM reviews
   GROUP BY book_id) AS SUB2 ON SUB1.id = SUB2.book_id
ORDER BY mean_rating DESC;
'''
exercise2 = pd.io.sql.read_sql(query, con=engine)
exercise2

,id,book_name,mean_rating,count_reviews
0,330,How to Be a Domestic Goddess: Baking and the A...,5.00,1.0
1,469,Neil Gaiman's Neverwhere,5.00,2.0
2,136,Captivating: Unveiling the Mystery of a Woman'...,5.00,2.0
3,972,Wherever You Go There You Are: Mindfulness Me...,5.00,2.0
4,642,The Big Bad Wolf (Alex Cross #9),5.00,2.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,202,Drowning Ruth,2.00,3.0
997,371,Junky,2.00,2.0
998,316,His Excellency: George Washington,2.00,2.0


Como podemos observar, tenemos una tabla con la calificación promedio y número de reseñas por cada libro junto con su titulo. Por ejemplo, el libro de `Angels Fall` tiene una calificación media de 5.0 y un total de reviews de 2. Esto nos puede ayudar a ver los libros mejor rankeados y cuantas reviews podemos encontrar de ellos.

### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [21]:
# Editorial que ha publicado mayor número de libros con más de 50 páginas
query = '''
SELECT SUB.publisher_id AS id,
       publishers.publisher AS publisher_name,
       COUNT(SUB.book_id) AS count_books
FROM
  (SELECT book_id,
          publisher_id
   FROM books
   WHERE num_pages > 50) AS SUB
INNER JOIN publishers ON publishers.publisher_id = SUB.publisher_id
GROUP BY id,
         publisher_name
ORDER BY count_books DESC
LIMIT 1
'''
exercise3 = pd.io.sql.read_sql(query, con=engine)
exercise3

,id,publisher_name,count_books
0,212,Penguin Books,42


De acuerdo a la consulta, podemos ver que la editorial Penguin Books es la que mayor número de libros ha publicado con 42 libros con más de 50 páginas. Un análisis posterior podria indicarnos el promedio de las reviews de los libros de la editorial

### Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [22]:
# Autor con calificacion promedio más alta de libros con al menos 50 calificaciones
query = '''
SELECT books.author_id,
       author,
       AVG(avg_rating) AS avg_ratings
FROM
  (SELECT books.book_id AS id_book,
          COUNT(rating_id) AS count_ratings,
          AVG(rating) AS avg_rating
   FROM ratings
   INNER JOIN books ON books.book_id = ratings.book_id
   GROUP BY id_book) AS SUB
INNER JOIN books ON books.book_id = SUB.id_book
INNER JOIN authors ON authors.author_id = books.author_id
WHERE count_ratings >= 50
GROUP BY books.author_id,
         author
ORDER BY avg_ratings DESC
LIMIT 1
'''

exercise4 = pd.io.sql.read_sql(query, con=engine)
exercise4

,author_id,author,avg_ratings
0,236,J.K. Rowling/Mary GrandPré,4.283844


Como podemos ver en la consulta, la autora J.K. Rowling es la autora con mejor calificacion promedio de sus libros con más de 50 calificaciones.

### Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [23]:
# Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros
query = '''
SELECT AVG(count_reviews) AS avg_reviews
FROM
  (SELECT username,
          count(rating) AS count_rating
   FROM ratings
   GROUP BY username) AS SUB
INNER JOIN
  (SELECT username,
          COUNT(text) AS count_reviews
   FROM reviews
   GROUP BY username) AS SUB1 ON SUB.username = SUB1.username
WHERE count_rating > 50
'''

exercise5 = pd.io.sql.read_sql(query, con=engine)
exercise5

,avg_reviews
0,24.333333


Podemos ver por la consulta, que el numero promedio de reseñas de texto que realizan usuarios que califican más de 50 libros es 24.3, minimo el 50% de los libros que han calificado.

## Conclusiones generales

En general, los resultados de nuestras consultas son los siguientes:
- El total de libros publicados despues del 1 de enero del 200 son 819.
- Se logro generar una tabla donde por cada libro se tiene su rating promedio y su número de reseñas de texto
- La editorial con mayor número de libros mayores a 50 páginas es Penguin Books con 42
- J.K. Rowling es la autora con mejor calificacion promedio (4.2) en libros con más de 50 calificaciones
- El número promedio de reseñas de texto que realizan usuarios que califican más de 50 libros es de 24.3, minimo el 50% de los libros que han calificado